In [ ]:
from TSP_Formulation_Methods import *
import numpy as np

distances = np.loadtxt("./data/matriz-rutas-granada")

# Global Parameters

N = 10
p = 4
L = 3
startNodes = [0,1,2]
endNodes = [9,8,7]

# Process Parameters

p = min(p, N-1)
startNode = min(startNode, N-1)
endNode = min(endNode, N-1)

reduced_distances = distances[:N,:N]/np.max(distances[:N,:N])